In [1]:
# ###import modules###
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
import sys
sys.path.append('../../myenv/lib/python3.7/site-packages')

In [3]:
class Time: # change class name to Time
    
    """Time stamp for code execution"""
        
    def get_current_time(self):
        now = datetime.datetime.now()
        return(now)
    
    def get_start_time(self):
        # Start time
        start_time = self.get_current_time()
        return(start_time)
       
    def get_end_time(self):
        # End time
        end_time = self.get_current_time()
        elapsed_time = end_time - self.get_start_time()
        return(end_time, elapsed_time)
        
    def print_start(self):
        print('--------Start Script--------')
        print('--------Start Time: ' + self.get_start_time().strftime('%Y-%m-%d %H:%M:%S') + '-------\n')

    def print_end(self):
        print('Total ' + str(self.get_end_time()[1].seconds) + ' [sec]')
        print('-----End Time : ' + self.get_end_time()[0].strftime('%Y-%m-%d %H:%M:%S') + ' ---------')
        print('-----END SCRIPT------') 

In [4]:
class Data():
    
    """Clean and transform data"""
    
    def find_first_col(self, df):
        for i in range(2):
            test = [col for col in df[df.columns[i]].values]
            try:
                any('北海' in s for s in test) or any('沖縄' in s for s in test)
                return(i)
            except:
               print('Warning: Not the starting column number.')
    
    def format_data(self, cleantext):
        # Converting string to list 
        res = cleantext.strip('][').split(', ') 
        return(res)

    def list_slice(self, S, step):
        return [S[i::step] for i in range(step)]
    
    def check_header(self, list_df):
        print("First value in list: " + list_df[0])
        if '国' in list_df[0]:
            list_df
        elif '感染' in list_df[0]:
            list_df.insert(0, '国・地域')
        else:
            print("Error: Starting value of data is neither 国, 感染")
        return(list_df)

    def check_row_num(self, list_df):
        #check if length of rows match
        num_rows = []
        check = [len(list_df[row]) for row in range(len(list_df))]
        if len(set(check)) == 1:
            print("Number of rows in lists are equal, ready to construct dataframe.")
        else:
            print("Error: Number of rows in list are unequal.")
            
    def create_df(self, list_df):
        df = pd.DataFrame()
        df['Country'] = list_df[0][1:]
        df['Infected'] = list_df[1][1:]
        df['Deaths'] = list_df[2][1:]
        print(df.head())
        return(df)
            
    def change_type(self, df, col_name, data_type):
        df[col_name] = df[col_name].astype(data_type)
        return(df)
        
    def drop_col(self, df, list_col_num):
        # Drop column
        for col_num in list_col_num:
            df = df.drop(df.columns[col_num], axis=1)
        return(df)
    
    def replace(self, df, col_name, target_list, replacement):
        for target in target_list:
            df[col_name] = df[col_name].str.replace(target, replacement)
        return(df)
    
    def fix_prefecture_name(self, df):
        test_df = df.to_dict('index')
        for pref in test_df.keys():
            if ('県' in pref or '東京都' in pref or '府' in pref):
                n_pref = pref
                print("Prefecture name already formatted")
            elif ('県' not in pref and '東京' not in pref and '大阪' not in pref and '京都' not in pref and '北海道' not in pref):
                n_pref = pref + '県'
            elif ('東京' in pref):
                n_pref = pref + '都'
            elif('大阪' in pref or '京都' in pref):
                n_pref = pref + '府'
            elif('北海道' in pref):
                n_pref = pref
            else:
                print("Warning: Prefecture name may not be formatted correctly.")
            test_df[pref]['pref'] = n_pref
        return(test_df)
        
    def check_data(self, ff_df, df):
        sample = ff_df.sample() #sample data from final df
        test_pref = sample['pref']
        check_df1 = df.loc[test_pref] #get data from original data
        for i in check_df1:
            if check_df1[i].values == sample[i].values:
                print('Values match:{}:{} = {}'.format(i, check_df1[i].values, sample[i].values))
            else:
                print('Error: Values do not match:{}:{} = {}'.format(i, check_df1[i].values, sample[i].values))

In [5]:
class File():
    
    """Access Files"""
    
    def append_value(self, dict_obj, key, value):
        # Check if key exist in dict or not
        if key in dict_obj.keys():
            dict_obj[key].append(value)
        else:
            dict_obj[key] = [value]

    def get_paths(self, mypath, ending, starting="2020_"):
        file_paths = {}
        for file in os.listdir(mypath):
            if file.startswith(starting) and file.endswith(tuple(ending)):
                group_key = file.rsplit('_')
                key = '_'.join(group_key[:3])
                self.append_value(file_paths, key, file)
            else:
                pass
        return(file_paths)
    
    def get_recent_file(self, get_paths_dict):
        d = [datetime.datetime.strptime(day, '%Y_%m_%d') for day in get_paths_dict.keys()]
        target = sorted(d)[-1]
        find_key = "{}_{}_{}".format(target.year, target.month, target.day)
        recent_file = csv_paths[find_key][0]
        return(recent_file)
            
    def export_table(self, table, file_name, file_type):
        # export individually
        path = '../../data/transformed/'
        
        if file_type == "csv":
            file_type = ".csv"
            table.to_csv(path + file_name + file_type)
        elif file_type == "txt":
            file_type = ".txt"
            with open(path + file_name + file_type, "w") as text_file:
                text_file.write(table)
        else:
            print('Error: Indicate data, filename, filetype - txt or csv')

In [14]:
def pcr_japan(mypath, csv_paths):
    data = Data()
    file = File()
    
    """国内における都道府県別のPCR検査陽性者数"""
    
    for key, value in csv_paths.items():
        frames = []
        for i, file_n in enumerate(value):
            print('data{}'.format(i) +  ": " + file_n)
            df = pd.read_csv(mypath + file_n, header=1)
            print("length of df: " + str(len(df)))

            ###Step2: Manipulate data###
            start = data.find_first_col(df)
            print(start)
            # Drop first column
            df = df.iloc[:, start:start+3]

            # Rename columns
            df.columns = ["Prefecture", "Positive", "Tested"]
            
            df = data.change_type(df, 'Prefecture', str)
            df = data.change_type(df, 'Positive', str)
            df = data.change_type(df, 'Tested', str)

            # Replace 
            df = data.replace(df, 'Prefecture', ['※','\n','\d+', ',,', '、', ' ', ',', '\(/～\）', '\(その他\)県', '\(～/\)', '⻑崎船その他県', '⻑崎船その他その他県', '合計\　県'], '')
            df = data.replace(df, 'Positive', [',', ' ', '\n', '\(208\)', '\(195\)', r"\(.*\)", '※', '名'], '')
            df = data.replace(df, 'Tested', [',', '※1', '※１', 'HP', 'nan', '※2', '-'], '')
            df = df.replace(r'^\s*$', np.nan, regex=True)
            df = df.reset_index() 
            
            print(df)

            # Drop last two rows
            df = df[df.Prefecture != '合計']
            df = df[df.Prefecture != 'その他']
            df = df[df.Prefecture != '⾧崎船その他']
            df = df[df.Prefecture != '長崎船その他']
            frames.append(df)

        result = pd.concat(frames)

        #check row length
        print("Length of df should be 47: " + str(len(result)))

        # change col type
        result = data.change_type(result, 'Positive', int)
        result = data.change_type(result, 'Tested', float)
        result = data.drop_col(result, [0])
        result = result.set_index('Prefecture')
        fix_pref = data.fix_prefecture_name(result)
        result = pd.DataFrame.from_dict(fix_pref, orient='index') 
        result = result.set_index('pref')
        
        #add percent col
        result['Percent'] = (result['Positive']/result["Tested"])*100
        result['Percent'] = result['Percent'].round(decimals=2)
        
        print(result.head())
        ###Step3: Export data###
        # export data to 'data/transformed/'
        file.export_table(result, '{}_corona_jp'.format(key), 'csv')
       

        
def corona_country(mypath, txt_paths):
    data = Data()
    file = File()
    
    """国別コロナ数"""

    for key, value in txt_paths.items():
        print('data-{}'.format(key) +  ": " + mypath + value)

        # Access file content
        file1 = open(mypath+value,"r") 
        clean_data = file1.readlines(0)[0]
        file1.close()

        # ###Step2: Manipulate data###
        list_df = data.format_data(clean_data)
        while "'名'" in list_df: list_df.remove("'名'")
        while "'※'" in list_df: list_df.remove("'※'")
        while "'（英王室属領）'" in list_df: list_df.remove("'（英王室属領）'")
        while "'ヘルツェゴビナ'" in list_df: list_df.remove("'ヘルツェゴビナ'")
        while "'バーブーダ'" in list_df: list_df.remove("'バーブーダ'")
        while "'ネービス'" in list_df: list_df.remove("'ネービス'")
        while "'クレナディーン諸島'" in list_df: list_df.remove("'クレナディーン諸島'")
        while "'及びクレナディーン諸島'" in list_df: list_df.remove("'及びクレナディーン諸島'")
        while "'（英領）'" in list_df: list_df.remove("'（英領）'")
        while "'カイコス諸島（英領）'" in list_df: list_df.remove("'カイコス諸島（英領）'")    

        list_df = data.check_header(list_df)
        list_df = data.list_slice(list_df[0:],3)

        data.check_row_num(list_df)
        df = data.create_df(list_df)

        # Replace 
        df = data.replace(df, 'Country', [r"[\"\',]"], '')
        df = data.replace(df, 'Infected', [r"[\"\',]", '名'], '')
        df = data.replace(df, 'Deaths', [r"[\"\',]", '名'], '')
        df = data.change_type(df, 'Infected', int)
        df = data.change_type(df, 'Deaths', int)
        df.head()

        ###Step3: Export data###
        # export data to 'data/transformed/'
        file.export_table(df, '{}_corona_country'.format(key), 'csv')
        
def population_jpn(mypath, path):
    data = Data()
    file = File()
    
    """国内における都道府県別の人口"""

    for key, value in path.items():
        frames = []
        for file_n in value:
            # read in data
            df = pd.read_csv(mypath+file_n, header=0)
            ###Step2: Manipulate data###
            # Drop first column
            df = df.drop(df.columns[0], axis=1)
            df = df.set_index('pref')
            # Replace 
            col_name = file_n.rsplit('_')[3:][0][:-4]
            df = data.replace(df, col_name, [','], '')

            # change col type
            if col_name == 'popDensity' or col_name == 'area':
                df = data.change_type(df, col_name, float)
            elif col_name == 'population':
                df = data.change_type(df, col_name, int)
            else:
                Print("Error: No such column name exist: {}".format(col_name))

            frames.append(df)

        final_df = pd.concat(frames, axis=1, join='inner')
        ff_df = final_df.reset_index()
        data.check_data(ff_df, df)
        print(final_df.dtypes)
        print(final_df.head())

        ###Step3: Export data###
        # export data to 'data/transformed/'
        file.export_table(final_df, '{}_population_jp'.format(key), 'csv')
        
def time_series_df(mypath, file_paths):
    data = Data()
    file = File()
    
    """Construct time series data"""
    
    frames = []
    for key, value in file_paths.items():
            # read in data
            print("Date of data: {}".format(key))
            df = pd.read_csv(mypath+value[0], header=0)
            ###Step2: Manipulate data###
#             df = df.drop(['Tested','Percent'], axis=1)  # Drop columns 'Positive', 'Tested', 'Percent'
            df = df.drop(['Positive', 'Percent'], axis=1)
            df = data.replace(df, 'pref', [','], '')
            print("Number of prefectures: {}".format(len(df['pref'])))
            df = df.set_index('pref')  # set index to prefectures
            d = datetime.datetime.strptime(key, '%Y_%m_%d')
            d = d.date() # convert datetime object to date object
            df.columns = ['{}'.format(d.isoformat())]
            frames.append(df)
    result = pd.concat(frames, axis=1, join='outer')
    df_transposed = result.T
    df_reset = df_transposed.reset_index()
    df_final=df_reset.rename(columns = {'index':'date'})
    df_final=df_final.sort_values('date')
    print("Number of prefectures: {}".format(len(df_final.columns)))
    print(df_final.columns)
    tmp = []
    for col in df_final.columns:
        tmp.append(str(col))
    print('check this: {}'.format(len(set(tmp))))
    
    # find missing dates 
    d = [datetime.datetime.strptime(day, '%Y-%m-%d') for day in df_final['date'].tolist()]
    date_set = set(d[0] + datetime.timedelta(x) for x in range((d[-1] - d[0]).days))
    missing = sorted(date_set - set(d))
    print("Missing data: {}".format(missing))

    ###Step3: Export data###
    # export data to 'data/transformed/'
#     file.export_table(df_final, 'corona_bytime_jp_today', 'csv')
    file.export_table(df_final, 'corona_bytime_jp_tested', 'csv')
    
def format_missing_data(mypath, filename):
    
    """Format Missing Data"""

    data = Data()
    file = File()
    
    df = pd.read_csv(mypath+filename, header=0)
    df = df.drop(['Unnamed: 0'], axis=1)  # Drop columns
    for i in range(len(df)):
        if df.iloc[i][1] == '北海道':
            df.iloc[i][1] = '北海道県'
        else:
            pass
    date_dict = {'date': []}    
    temp_df = {
    '滋賀県':[],'京都府':[],'大阪府':[],'兵庫県':[],'奈良県':[],
    '和歌山県':[],'鳥取県':[],'島根県':[],'岡山県':[],'広島県':[],
    '山口県':[],'徳島県':[],'香川県':[],'愛媛県':[],'高知県':[],
    '福岡県':[],'佐賀県':[],'長崎県':[],'熊本県':[],'大分県':[],
    '宮崎県':[],'鹿児島県':[],'沖縄県':[],'北海道':[],'青森県':[],
    '岩手県':[],'宮城県':[],'秋田県':[],'山形県':[],'福島県':[],
    '茨城県':[],'栃木県':[],'群馬県':[],'埼玉県':[],'千葉県':[],
    '東京都':[],'神奈川県':[],'新潟県':[],'富山県':[],'石川県':[],
    '福井県':[],'山梨県':[],'長野県':[],'岐阜県':[],'静岡県':[],
    '愛知県':[],'三重県':[],
    }
    for i in range(len(df)):
        date_dict['date'].append(df.iloc[i][0])
        for key in temp_df:
            if df.iloc[i][1] == key:
                temp_df[key].append(1)
            else:
                temp_df[key].append(0)
                
    date_dict.update(temp_df)
    final_df = pd.DataFrame.from_dict(date_dict)
    final_df = final_df[final_df.date != '調査中']
    final_df = data.replace(final_df, 'date', ["/"], "-")
    final_df['date'] = pd.to_datetime(final_df['date'])
    print('Number of prefectures: {}'.format(len(final_df.columns)))
    
    return(final_df)

def concat_missing_data(org_data, missing_data):
    
    org_df = pd.read_csv(org_data, header=0)
    org_df = org_df.drop(['Unnamed: 0'], axis=1)  # Drop columns
    # convert the 'Date' column to datetime format
    org_df['date'] = pd.to_datetime(org_df['date'])
    mis_df = pd.read_csv(missing_data, header=0)
#     mis_df = mis_df.drop(['Unnamed: 0'], axis=1)  # Drop columns
    mis_df['date'] = pd.to_datetime(mis_df['date'])

    frames = [org_df, mis_df]
    result = pd.concat(frames)
    print('Length of colums: {}'.format(len(result.columns)))

    result = result.sort_values(by=['date'])
    return(result)

In [15]:
class Main():
    
    """Transform data"""
    
    time = Time()
    time.print_start()
    
    data = Data()
    file = File()
    
    ###Step1: Read in data###
    mypath = '../../data/raw/'
   
#     #国別コロナ数
#     txt_paths = file.get_paths(mypath, ".txt")
#     corona_country(mypath, txt_paths)
    
#     #国内における都道府県別の人口
#     path = file.get_paths(mypath, ["popDensity.csv", "area.csv", "population.csv"])
#     population_jpn(mypath, path)

#     国内における都道府県別のPCR検査陽性者数
#     csv_paths = file.get_paths(mypath, ["corona_jp_1.csv", "corona_jp_2.csv"], starting="2020_12_31")
#     pcr_japan(mypath, csv_paths)

#     #Get rest of pcr missing data
#     mypath = '/Users/rurikoimai/Desktop/corona/data/missing_csv/'
#     file_dict = file.get_paths(mypath, ["corona_jp_1.csv", "corona_jp_2.csv"], starting="2020_")
#     pcr_japan(mypath, file_dict)
    
#     #concat population and pcr df
#     file = File()
#     pop_data = '../../data/transformed/2020_10_1_population_jp.csv'
#     pcr_data = '../../data/transformed/corona_jp/'
#     csv_paths = file.get_paths(pcr_data, ["corona_jp.csv"], starting="2020_12_")
#     recent_file = file.get_recent_file(csv_paths)
#     pop_df = pd.read_csv(pop_data, header=0)
#     pcr_df = pd.read_csv(pcr_data+recent_file, header=0)
#     pop_df = pop_df.set_index('pref')
#     pcr_df = pcr_df.set_index('pref')
#     frames = [pcr_df, pop_df]
#     result = pd.concat(frames, axis=1)
#     file.export_table(result, 'pcr_pop_corona_jp', 'csv')
    
    #Construct time series data###
    mypath = '../../data/transformed/corona_jp/'
    file_paths = file.get_paths(mypath, ".csv")
    time_series_df(mypath, file_paths)
    
#     #Format Missing Data
#     mypath = '../../data/'
#     filename = '2020_1_15_corona_jp_1.csv'
#     missing_df = format_missing_data(mypath, filename)
#     missing_df = missing_df.set_index('date')
#     # add row with next row
#     for col in missing_df.columns:
#         for i in range(0, len(missing_df)-1):
#             missing_df[col].values[i + 1] = missing_df[col].values[i] + missing_df[col].values[i + 1]

#     file.export_table(missing_df, 'missing_data_corona_jp', 'csv')
    
#     #Concat missing data to original data
#     org_data = '../../data/transformed/corona_bytime_jp_v0.1.csv'
#     missing_data = '../../data/transformed/missing_data_corona_jp.csv'
#     result = concat_missing_data(org_data, missing_data)
#     file.export_table(result, 'corona_bytime_jp_v0.2', 'csv')

#     #Find missing dates 
#     d = [day for day in result['date'].tolist()]
#     date_set = set(d[0] + datetime.timedelta(x) for x in range((d[-1] - d[0]).days))
#     missing = sorted(date_set - set(d))
#     print("Missing data: {}".format(missing))

#     #Update time series data###
#     mypath = '../../data/transformed/'
#     file_paths = file.get_paths(mypath, "corona_jp.csv", starting='2020_12_31')
#     time_series_df(mypath, file_paths)
#     time_data = '../../data/transformed/corona_bytime_jp_updated.csv'
#     recent_data = '../../data/transformed/corona_bytime_jp_today.csv'
#     result = concat_missing_data(time_data, recent_data)
#     file.export_table(result, 'corona_bytime_jp_updated', 'csv')


    
    #国内の自殺者数 

    time.print_end()
    
if __name__ == '__main__':
    Main()

--------Start Script--------
--------Start Time: 2020-12-31 21:23:47-------

Date of data: 2020_7_19
Number of prefectures: 47
Date of data: 2020_6_29
Number of prefectures: 47
Date of data: 2020_5_16
Number of prefectures: 47
Date of data: 2020_3_30
Number of prefectures: 47
Date of data: 2020_7_20
Number of prefectures: 47
Date of data: 2020_12_7
Number of prefectures: 47
Date of data: 2020_5_6
Number of prefectures: 47
Date of data: 2020_6_10
Number of prefectures: 47
Date of data: 2020_3_27
Number of prefectures: 47
Date of data: 2020_5_12
Number of prefectures: 47
Date of data: 2020_8_4
Number of prefectures: 47
Date of data: 2020_4_22
Number of prefectures: 47
Date of data: 2020_3_23
Number of prefectures: 47
Date of data: 2020_5_2
Number of prefectures: 47
Date of data: 2020_6_14
Number of prefectures: 47
Date of data: 2020_12_3
Number of prefectures: 47
Date of data: 2020_7_24
Number of prefectures: 48
Date of data: 2020_10_16
Number of prefectures: 47
Date of data: 2020_8_30
N

Number of prefectures: 47
Date of data: 2020_7_16
Number of prefectures: 47
Date of data: 2020_6_18
Number of prefectures: 47
Date of data: 2020_6_2
Number of prefectures: 47
Date of data: 2020_5_27
Number of prefectures: 47
Date of data: 2020_7_28
Number of prefectures: 47
Date of data: 2020_5_30
Number of prefectures: 47
Date of data: 2020_11_3
Number of prefectures: 47
Date of data: 2020_4_17
Number of prefectures: 48
Date of data: 2020_8_8
Number of prefectures: 47
Date of data: 2020_6_21
Number of prefectures: 47
Date of data: 2020_3_16
Number of prefectures: 47
Date of data: 2020_7_11
Number of prefectures: 47
Date of data: 2020_11_7
Number of prefectures: 47
Date of data: 2020_4_13
Number of prefectures: 47
Date of data: 2020_5_23
Number of prefectures: 47
Date of data: 2020_6_6
Number of prefectures: 47
Date of data: 2020_7_15
Number of prefectures: 47
Date of data: 2020_3_12
Number of prefectures: 47
Date of data: 2020_6_25
Number of prefectures: 47
Date of data: 2020_11_17
Nu